In [ ]:
import pandas as pd
import numpy as np
import os
import random
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
train_directory = 'D:/Assignment/CIFAR-10-images-master/train'
catagories = []
for c in os.listdir(train_directory):
    catagories.append(c)
print(catagories)

In [ ]:
train_Data = []
for c in catagories:
    path = os.path.join(train_directory,c)
    class_num = catagories.index(c)
    for img in tqdm(os.listdir(path)):
        img_arr = cv2.imread(os.path.join(path,img))
        train_Data.append([img_arr, class_num])
print(len(train_Data))

In [ ]:
random.shuffle(train_Data)
plt.figure(figsize=(20,10))

for i in range(50):
    plt.subplot(5,10,i+1)
    plt.imshow(train_Data[i][0])
    plt.xlabel(catagories[train_Data[i][1]])
    plt.xticks([])
    plt.yticks([])
    if i==50:
        break
plt.show()

In [ ]:
f0 = train_Data[0:1000]
f1 = train_Data[1000:2000]
f2 = train_Data[2000:3000]
f3 = train_Data[3000:4000]
f4 = train_Data[4000:5000]

In [ ]:
class NearestNeighbourClassifier():

    def __init__(self,k,distancemetric="l2"):
        self.K = k
        self.trainSet = None
        self.distanceMetric = distancemetric.lower()

    def fit(self,train):
        self.trainSet = train


    def __L2_distance(self, train_img,test_img):
        temp = train_img - test_img
        dis = np.sqrt(np.sum(np.square(temp)))
        return dis

    def __distanceCalculator(self,validate):
        accuracy = 0
        for vidx in range(len(validate)):
            distnaceList = []
            cnt = 0
            for tidx in range(len(self.trainSet)):
                d = 0
                if self.distanceMetric == 'l2':
                    d = self.__L2_distance(self.trainSet[tidx][0],validate[vidx][0])
                distnaceList.append((d,self.trainSet[tidx][1]))

            distnaceList.sort(key=(lambda a: a[0]))
            labels = [v[1] for v in distnaceList[:self.K]]
            label = max(set(labels), key = labels.count)

            if label == validate[vidx][1]:
                cnt+=1

            accuracy += cnt
        return accuracy, (accuracy/len(validate))

    def predict(self,validate):
        return self.__distanceCalculator(validate)

    def plot_Accuracies(fold,k_choices,accuracies):
    best = [0,0]
    for k in k_choices:
        acc = accuracies[k]
        if best[1]< max(acc):
            best[0]=k
            best[1]=max(acc)
        plt.scatter([k] * len(acc), acc)


    accuracies_mean = np.array([np.mean(v) for k,v in sorted(accuracies.items())])
    accuracies_std = np.array([np.std(v) for k,v in sorted(accuracies.items())])
    plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
    plt.title(f"Cross-validation   fold={fold}   K={best[0]}   Accuracy = {best[1]}")
    plt.xlabel('k')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
k_choices = [3, 5, 8, 10, 12, 15, 20]

In [ ]:
accuracis = {}
nfolds = 5
for k in k_choices:
    accuracis[k]=[]
for i in range(nfolds):
    if i==0:
        trainSet = f1+f2+f3+f4
        validate = f0
    elif i==1:
        trainSet = f0+f2+f3+f4
        validate = f1
    elif i==2:
        trainSet = f1+f0+f3+f4
        validate = f2
    elif i==3:
        trainSet = f1+f2+f0+f4
        validate = f3
    elif i==4:
        trainSet = f1+f2+f3+f0
        validate = f4

    for k in k_choices:

        classifier = NearestNeighbourClassifier(k,'l2')
        classifier.fit(trainSet)
        cnt, acc = classifier.predict(validate)

        accuracis[k].append(acc)

In [ ]:
plot_Accuracies(5,k_choices,accuracis)